cleaning up text before tokenizer will produce the best results

passing HTML as text produces chaos

In [2]:
curl -XGET 'localhost:9200/_analyze?tokenizer=standard&pretty' -H 'Content-Type: application/json' -d'
{
  "text": "<p>Some d&eacute;j&agrave; vu <a href=\"http://somedomain.com>\">website</a>" 
}
'

{
  "tokens" : [
    {
      "token" : "p",
      "start_offset" : 1,
      "end_offset" : 2,
      "type" : "<ALPHANUM>",
      "position" : 0
    },
    {
      "token" : "Some",
      "start_offset" : 3,
      "end_offset" : 7,
      "type" : "<ALPHANUM>",
      "position" : 1
    },
    {
      "token" : "d",
      "start_offset" : 8,
      "end_offset" : 9,
      "type" : "<ALPHANUM>",
      "position" : 2
    },
    {
      "token" : "eacute",
      "start_offset" : 10,
      "end_offset" : 16,
      "type" : "<ALPHANUM>",
      "position" : 3
    },
    {
      "token" : "j",
      "start_offset" : 17,
      "end_offset" : 18,
      "type" : "<ALPHANUM>",
      "position" : 4
    },
    {
      "token" : "agrave",
      "start_offset" : 19,
      "end_offset" : 25,
      "type" : "<ALPHANUM>",
      "position" : 5
    },
    {
      "token" : "vu",
      "start_offset" : 27,
      "end_offset" : 29,
      "type" : "<ALPHANUM>",
      "position" : 6
    },
    {
      "token" : "

character filters can be used to preprocess text before it is analyzed. Use html filter in this example

In [17]:
curl -XDELETE 'localhost:9200/my_index'

{"acknowledged":true}

In [18]:
curl -XPUT 'localhost:9200/my_index?pretty' -H 'Content-Type: application/json' -d'
{
    "settings": {
        "analysis": {
            "analyzer": {
                "my_html_analyzer": {
                    "tokenizer":     "standard",
                    "char_filter": [ "html_strip" ]
                }
            }
        }
    }
}
'

{
  "acknowledged" : true,
  "shards_acknowledged" : true
}


In [20]:
curl -XGET 'localhost:9200/my_index/_analyze?pretty' -H 'Content-Type: application/json' -d'
{
  "analyzer": "my_html_analyzer",
  "text": "<p>Some d&eacute;j&agrave; vu <a href=\"http://somedomain.com>\">website</a>"  
}
'

{
  "tokens" : [
    {
      "token" : "Some",
      "start_offset" : 3,
      "end_offset" : 7,
      "type" : "<ALPHANUM>",
      "position" : 0
    },
    {
      "token" : "déjà",
      "start_offset" : 8,
      "end_offset" : 26,
      "type" : "<ALPHANUM>",
      "position" : 1
    },
    {
      "token" : "vu",
      "start_offset" : 27,
      "end_offset" : 29,
      "type" : "<ALPHANUM>",
      "position" : 2
    },
    {
      "token" : "website",
      "start_offset" : 63,
      "end_offset" : 74,
      "type" : "<ALPHANUM>",
      "position" : 3
    }
  ]
}


sometimes you'll want to map characters (in the case of special apostrophe characters) to a specific character. Use a custom char filter

In [21]:
curl -XDELETE 'localhost:9200/my_index'

{"acknowledged":true}

In [22]:
curl -XPUT 'localhost:9200/my_index?pretty' -H 'Content-Type: application/json' -d'
{
  "settings": {
    "analysis": {
      "char_filter": { 
        "quotes": {
          "type": "mapping",
          "mappings": [ 
            "\\u0091=>\\u0027",
            "\\u0092=>\\u0027",
            "\\u2018=>\\u0027",
            "\\u2019=>\\u0027",
            "\\u201B=>\\u0027"
          ]
        }
      },
      "analyzer": {
        "quotes_analyzer": {
          "tokenizer":     "standard",
          "char_filter": [ "quotes" ] 
        }
      }
    }
  }
}
'

{
  "acknowledged" : true,
  "shards_acknowledged" : true
}


In [23]:
curl -XGET 'localhost:9200/my_index/_analyze?pretty' -H 'Content-Type: application/json' -d'
{
  "analyzer": "quotes_analyzer",
  "text": "You’re my ‘favorite’ M‛Coy"  
}
'

{
  "tokens" : [
    {
      "token" : "You're",
      "start_offset" : 0,
      "end_offset" : 6,
      "type" : "<ALPHANUM>",
      "position" : 0
    },
    {
      "token" : "my",
      "start_offset" : 7,
      "end_offset" : 9,
      "type" : "<ALPHANUM>",
      "position" : 1
    },
    {
      "token" : "favorite",
      "start_offset" : 11,
      "end_offset" : 19,
      "type" : "<ALPHANUM>",
      "position" : 2
    },
    {
      "token" : "M'Coy",
      "start_offset" : 21,
      "end_offset" : 26,
      "type" : "<ALPHANUM>",
      "position" : 3
    }
  ]
}
